In [1]:
import nltk
import numpy as np
import pandas as pd
import connect_aws_db as cadb
from textblob import TextBlob as tb

In [2]:
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
engine = cadb.connect_aws_db(write_unicode=True)

### Read in BF reviews

In [4]:
cmd = "SELECT review_rating, review_text FROM bf_reviews"

In [5]:
bfdf = pd.read_sql_query(cmd, engine)

In [6]:
bfdf

,review_rating,review_text
0,5,Really nice property. Great walking areas for ...
1,5,We've stayed at this La Quinta several times w...
2,5,This place was awesome!! The entire staff was ...
3,5,We've stayed at this hotel two times with our ...
4,4,My room while a bit small was very clean and t...
5,3,The big gentleman who manages the front desk i...
6,1,"I was driving from So Cal to Houston, TX for t..."
7,2,the hotel was fine but it is all hard scape ar...
8,4,Hotel has a grass area and dog potty station. ...
9,3,My room in general was in need of a lot of gen...


In [7]:
bfreviews = ('').join(bfdf['review_text'].values)

In [8]:
bfreviews[:500]

u"Really nice property. Great walking areas for the pets. Centrally located. A lot of swimming pools to use. A water park for the kids. Pets are not allowed in the water park of course. The casita room was nice, but there was no table, making it hard to eat anything with the dogs there. Casita rooms are small. Overall, very nice!We've stayed at this La Quinta several times with our two mini schnauzers. The staff is all very friendly. Rooms were clean but not overly fancy - we felt like we didn't n"

### Read in Yelp reviews

In [10]:
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

In [11]:
bftokens = nltk.word_tokenize(bfreviews)

In [12]:
len(bftokens)

1042

In [26]:
stemmer = PorterStemmer()

In [29]:
bfstemmed = []

In [30]:
for item in bftokens:
    bfstemmed.append(stemmer.stem(item))

In [31]:
len(bfstemmed)

983

In [33]:
bftext = nltk.Text(bfstemmed)

In [34]:
bfwords = [w.lower() for w in bftext]

In [35]:
bfvocab = sorted(set(bfwords))

In [36]:
len(bfvocab)

408

### TFIDF on BF Reviews Using TextBlobs

In [13]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
document1 = tb("""Python is a 2000 made-for-TV horror movie directed by Richard
Clabaugh. The film features several cult favorite actors, including William
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy,
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean
Whalen. The film concerns a genetically engineered snake, a python, that
escapes and unleashes itself on a small town. It includes the classic final
girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles,
 California and Malibu, California. Python was followed by two sequels: Python
 II (2002) and Boa vs. Python (2004), both also made-for-TV films.""")

document2 = tb("""Python, from the Greek word (πύθων/πύθωνας), is a genus of
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are
recognised.[2] A member of this genus, P. reticulatus, is among the longest
snakes known.""")

document3 = tb("""The Colt Python is a .357 Magnum caliber revolver formerly
manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
It is sometimes referred to as a "Combat Magnum".[1] It was first introduced
in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
Colt Python targeted the premium revolver market segment. Some firearm
collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy
Thompson, Renee Smeets and Martin Dougherty have described the Python as the
finest production revolver ever made.""")

bloblist = [document1, document2, document3]
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))


### TFIDF on BF Reviews Using SciKitLearn

The number of words in the bringfido sample corpus:

In [37]:
len(bfwords)

983

In [54]:
count_vect = CountVectorizer()
count_vect.fit_transform(bfwords)

<983x379 sparse matrix of type '<type 'numpy.int64'>'
	with 898 stored elements in Compressed Sparse Row format>

In [55]:
bf_train_counts[0]

<1x379 sparse matrix of type '<type 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [56]:
print('vocabulary: {}'.format(count_vect.vocabulary))

vocabulary: None


In [40]:
tfidf_transformer = TfidfTransformer()
bf_train_tfidf = tfidf_transformer.fit_transform(bf_train_counts)
bf_train_tfidf.shape

(983, 379)

In [42]:
tf = TfidfVectorizer(analyzer='word', stop_words='english')

In [43]:
bftfidf_matrix = tf.fit_transform(bfwords)

In [44]:
bf_feature_names = tf.get_feature_names()

In [48]:
#bf_feature_names[0:50]

In [50]:
bfdense = bftfidf_matrix.todense()

### TFIDF on Yelp Reviews

First I want to import all of the Yelp review text, and perform a TFIDF on those reviews:

In [17]:
cmd = "SELECT review_rating, review_text FROM yelp_reviews"

In [18]:
yelpdf = pd.read_sql_query(cmd, engine)

In [19]:
len(yelpdf)

6263

In [20]:
yelpreviews = ('').join(yelpdf['review_text'].values)
yelptokens = nltk.wordpunct_tokenize(yelpreviews)
yelptext = nltk.Text(yelptokens)
yelpwords = [w.lower() for w in yelptext]

In [21]:
yelp_count_vect = CountVectorizer()
yelp_train_counts = yelp_count_vect.fit_transform(yelpwords)
yelp_train_counts.shape


(1244957, 22909)

In [22]:
yelp_tfidf_transformer = TfidfTransformer()
yelp_train_tfidf = yelp_tfidf_transformer.fit_transform(yelp_train_counts)
yelp_train_tfidf.shape

(1244957, 22909)

In [23]:
#from sklearn.metrics import jaccard_similarity_score

In [66]:
#jaccard_similarity_score(bf_train_tfidf, yelp_train_tfidf)

### Examples

In [22]:
path = './tf-idf'
token_dict = {}


def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [24]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)


In [ ]:
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        print "fname=", fname
        with open(fname) as pearl:
            text = pearl.read()
            token_dict[f] = text.lower().translate(None, string.punctuation)

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

str = 'all great and precious things are lonely.'
response = tfidf.transform([str])
print response

feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print feature_names[col], ' - ', response[0, col]

In [43]:
#twenty_train.data

In [ ]:
yelp_count_vect = CountVectorizer()
yelp_train_counts = yelp_count_vect.fit_transform(twenty_train.data)
X_train_counts.shape


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape


(2257, 35788)

In [28]:
X_train_counts[0:3, 0:3]

<3x3 sparse matrix of type '<type 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>